# Pre-Processing

In [141]:
import numpy as np
import os
import pandas as pd
import sqlite3

from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Get the root directory (INTA6450_Enron folder)
# Different than .py files __file__ attribute is not available in Jupyter notebooks
root_dir = os.path.dirname(os.path.dirname(os.getcwd()))

print(root_dir)

/Users/nmacdonald/projects


In [4]:
# Connect to the database (or create it if it doesn't exist)
connection = sqlite3.connect(f"{os.path.dirname(os.getcwd())}/emails.db")

# Create a cursor object to execute SQL commands
cursor = connection.cursor()

# Load the dataframe from the SQLite database
emails_df = pd.read_sql_query("SELECT * FROM emails", connection)

# Close the connection
connection.close()

# Show email data
emails_df.head()

,text,message_id,date,from,to,subject,cc,bcc,mime-version,content-type,content-transfer-encoding,x-from,x-to,x-cc,x-bcc,folder,origin,filename,priority
0,---------------------- Forwarded by Rika Imai/...,<88180.1075863689140.JavaMail.evans@thyme>,"Tue, 8 May 2001 08:37:00 -0700 (PDT)",rika.imai@enron.com,"john.forney@enron.com, mike.carson@enron.com, ...",4 Month Rolling Forecast,,,1.0,text/plain; charset=ANSI_X3.4-1968,,Rika Imai,"John M Forney, Mike Carson, Clint Dean, Doug G...",,,\Rob_Benson_Jun2001\Notes Folders\Notes inbox,Benson-R,rbenson.nsf,normal
1,great,<4460514.1075857469666.JavaMail.evans@thyme>,"Wed, 21 Jun 2000 02:01:00 -0700 (PDT)",hunter.shively@enron.com,richard.tomaski@enron.com,Re: Jim Simpson,,,1.0,text/plain; charset=us-ascii,,Hunter S Shively,Richard Tomaski,,,\Hunter_Shively_Jun2001\Notes Folders\Sent,Shively-H,hshivel.nsf,normal
2,"oohh la la. who was your ""friend""? did you g...",<2160301.1075858147494.JavaMail.evans@thyme>,"Wed, 16 Aug 2000 03:03:00 -0700 (PDT)",matthew.lenhart@enron.com,shelliott@dttus.com,Re: Re[2]:,,,1.0,text/plain; charset=us-ascii,,Matthew Lenhart,Shirley Elliott <shelliott@dttus.com> @ ENRON,,,\Matthew_Lenhart_Jun2001\Notes Folders\Sent,Lenhart-M,mlenhar.nsf,normal
3,\nAttached are the two files with this week's ...,<22847680.1075863611080.JavaMail.evans@thyme>,"Wed, 15 Aug 2001 05:46:47 -0700 (PDT)",rika.imai@enron.com,"russell.ballato@enron.com, hicham.benjelloun@e...",FW: Nuclear Rolling Forecast,,,1.0,text/plain; charset=us-ascii,,"Imai, Rika </O=ENRON/OU=NA/CN=RECIPIENTS/CN=RI...","Ballato, Russell </O=ENRON/OU=NA/CN=RECIPIENTS...",,,"\ExMerge - Benson, Robert\Inbox\Large Messages",BENSON-R,rob benson 6-25-02.PST,normal
4,lm:\nWhat are your thoughts going forward........,<15012282.1075852957298.JavaMail.evans@thyme>,"Wed, 3 Oct 2001 00:35:05 -0700 (PDT)",jennifer.fraser@enron.com,larry.may@enron.com,hello,,,1.0,text/plain; charset=us-ascii,,"Fraser, Jennifer </O=ENRON/OU=NA/CN=RECIPIENTS...","May, Larry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Lm...",,,\LMAY2 (Non-Privileged)\Inbox,May-L,LMAY2 (Non-Privileged).pst,normal


In [5]:
emails_df["text"].iloc[0]

'---------------------- Forwarded by Rika Imai/NA/Enron on 05/08/2001 03:37 PM \n---------------------------\n\n\n"Dan Salter" <dsalter@hgp-inc.com> on 05/08/2001 02:42:13 PM\nPlease respond to "Dan Salter" <dsalter@hgp-inc.com>\nTo: "Rogers Herndon" <Rogers.Herndon@enron.com>, "Rika Imai" \n<Rika.Imai@enron.com>, "Lloyd Will" <Lloyd.Will@enron.com>, "Kevin Presto" \n<Kevin.M.Presto@enron.com>, "Harry Aurora" <harora@enron.com>, "Fletch Sturm" \n<Fsturm@enron.com>, "Dana Davis" <Mark.Dana.Davis@enron.com>, "Doug \nGilbert-Smith" <Doug.Gilbert-Smith@enron.com>\ncc:  \n\nSubject: 4 Month Rolling Forecast\n\n\n\nAttached are the two files for this week\'s nuclear four month  rolling \nforecast.\n?\nDan\n - 010508 update.doc\n - 010508 update.xls\n'

# Text Extraction

In [7]:
# def extract_text(text):
#     # Remove HTML tags
#     text = BeautifulSoup(text, "html.parser").get_text() if pd.notnull(text) else ""
#     # Remove URLs and email addresses
#     text = re.sub(r"\S+@\S+", "", text)  # Remove email addresses
#     text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
#     return text


def extract_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text() if pd.notnull(text) else ""
    # Remove email addresses
    text = re.sub(r"\S+@\S+", "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove newline characters
    text = re.sub(r"\n", " ", text)
    # Remove special characters and punctuation
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

stop_words = set(stopwords.words("english"))
df = emails_df.copy()
df["processed_text"] = df["text"].apply(extract_text)
df["processed_text"]

/var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/ipykernel_78937/2121749081.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() if pd.notnull(text) else ""
/var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/ipykernel_78937/2121749081.py:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() if pd.notnull(text) else ""


0         forwarded rika imainaenron pm dan salter pm pl...
1                                                     great
2         oohh la la friend guys read stories together g...
3         attached two files weeks nuclear rolling forec...
4         lm thoughts going forward also seeing winter t...
                                ...                        
251729    heres picture august th retirement ceremony ho...
251730                       version shortened presentation
251731    may aware substantial revisions us tax code re...
251732    stephen rick shapiro revised allocation enron ...
251733    message sent pjmcustomerinfo mailing list pjms...
Name: processed_text, Length: 251734, dtype: object

In [8]:
emails_df["newline_count"] = emails_df["text"].str.count("\n")

In [17]:
# Calculate total number of emails and total newline count
total_emails = len(emails_df)
total_newlines = emails_df["text"].str.count("\n").sum()

# Count email addresses in each email body and calculate total email addresses
emails_df["email_count"] = emails_df["text"].str.count(r"\S+@\S+")
total_email_addresses = emails_df["email_count"].sum()

# Count URLs in each email body
emails_df["url_count"] = emails_df["text"].str.count(r"http\S+|www\S+")
total_urls = emails_df["url_count"].sum()

# Count special characters and punctuation in each email body
emails_df["special_char_count"] = emails_df["text"].str.count(r"[^a-zA-Z\s]")
total_special_chars = emails_df["special_char_count"].sum()

# Count words in each email body
emails_df["word_count"] = emails_df["text"].str.findall(r"\b\w+\b").str.len()
total_words = emails_df["word_count"].sum()

# Count sentences in each email body (heuristic based on sentence-ending punctuation)
emails_df["sentence_count"] = emails_df["text"].str.count(r"[.!?]")
total_sentences = emails_df["sentence_count"].sum()

# Create the new DataFrame with newline counts
newline_counts_df = emails_df[["text"]].copy()

# Add counts into df
newline_counts_df["New Lines \\n"] = emails_df["text"].str.count("\n")
newline_counts_df["Email Count"] = emails_df["email_count"]
newline_counts_df["URL Count"] = emails_df["url_count"]
newline_counts_df["Special Char Count"] = emails_df["special_char_count"]
newline_counts_df["Word Count"] = emails_df["word_count"]
newline_counts_df["Sentence Count"] = emails_df["sentence_count"]

# Create a new row for the totals and insert it at the top of the DataFrame
totals_row = pd.DataFrame(
    {
        "text": ["Total Emails"],
        "New Lines \\n": [total_newlines],
        "Email Count": [total_email_addresses],
        "URL Count": [total_urls],
        "Special Char Count": [total_special_chars],
        "Word Count": [total_words],
        "Sentence Count": [total_sentences],
    }
)
newline_counts_df = pd.concat([totals_row, newline_counts_df], ignore_index=True)

# Add total emails count as the first row
# newline_counts_df.at[0, "text"] = f"Total Emails: {total_emails}"

newline_counts_df.head()  # Display the first few rows to verify

,text,New Lines \n,Email Count,URL Count,Special Char Count,Word Count,Sentence Count
0,Total Emails,10411219,924798,231192,50562873,73075774,6494550
1,---------------------- Forwarded by Rika Imai/...,23,10,0,186,114,22
2,great,0,0,0,0,1,0
3,"oohh la la. who was your ""friend""? did you g...",1,0,0,8,29,6
4,\nAttached are the two files with this week's ...,3,0,0,2,13,1


In [23]:
# Count the number of unique email addresses
unique_email_addresses = emails_df["from"].nunique()

# Count the number of emails from each email address
emails_from_each_address = emails_df["from"].value_counts().reset_index()
emails_from_each_address.columns = ["Sender", "Email Count"]

# Add a summary row at the top with the total unique email addresses
summary_row = pd.DataFrame(
    {"Sender": ["Total Unique Senders"], "Email Count": [unique_email_addresses]}
)

# Concatenate the summary row with the email counts
email_counts_df = pd.concat([summary_row, emails_from_each_address], ignore_index=True)

# Display the first few rows of the new DataFrame to verify
email_counts_df.head(32)

,Sender,Email Count
0,Total Unique Senders,20328
1,jeff.dasovich@enron.com,5498
2,kay.mann@enron.com,4868
3,sara.shackleton@enron.com,4443
4,tana.jones@enron.com,4152
5,vince.kaminski@enron.com,3935
6,pete.davis@enron.com,3921
7,chris.germany@enron.com,3620
8,matthew.lenhart@enron.com,2762
9,debra.perlingiere@enron.com,2439


In [58]:
# Filter for email addresses that are not from @enron.com
# Clean up 'from' column by removing any trailing '>'
emails_df["from_cleaned"] = emails_df["from"].str.replace(r">$", "", regex=True)

non_enron_emails = emails_df["from"][
    ~emails_df["from_cleaned"].str.endswith("enron.com", na=False)
]
enron_emails = emails_df["from_cleaned"][emails_df["from"].str.endswith("enron.com", na=False)]

# Count the number of unique non-Enron email addresses
unique_non_enron_count = non_enron_emails.nunique()
unique_enron_count = enron_emails.nunique()

# Count the total number of emails from non-Enron email addresses
total_non_enron_emails = non_enron_emails.count()
total_enron_emails = enron_emails.count()

# Display the results
print(f"Number of unique non-Enron email addresses: {unique_non_enron_count}")
print(f"Total number of emails from non-Enron addresses: {total_non_enron_emails}")
print(f"Number of unique Enron email addresses: {unique_enron_count}")
print(f"Total number of emails from Enron addresses: {total_enron_emails}")

Number of unique non-Enron email addresses: 13830
Total number of emails from non-Enron addresses: 53789
Number of unique Enron email addresses: 6480
Total number of emails from Enron addresses: 196568


In [101]:
# Filter for Enron senders and count the number of emails from each
enron_sender_counts = emails_df["from_cleaned"][
    emails_df["from_cleaned"].str.endswith("enron.com", na=False)
].value_counts()

employee_emails_sent = enron_sender_counts.reset_index()
# Rename a column in place
employee_emails_sent.rename(columns={"from_cleaned": "Sender", "count": "Count"}, inplace=True)

employee_emails_sent.head(10)

,Sender,Count
0,jeff.dasovich@enron.com,5498
1,kay.mann@enron.com,4868
2,sara.shackleton@enron.com,4443
3,tana.jones@enron.com,4152
4,vince.kaminski@enron.com,3935
5,pete.davis@enron.com,3921
6,chris.germany@enron.com,3620
7,matthew.lenhart@enron.com,2762
8,debra.perlingiere@enron.com,2439
9,gerald.nemec@enron.com,2230


In [50]:
# Extract domains from non-Enron email addresses
non_enron_domains = non_enron_emails.str.extract(r"@(.+)$")[
    0
]  # Extracts the domain part after '@'


# Count the occurrences of each domain
domain_counts = non_enron_domains.value_counts()

# Display the top 10 most common non-Enron domains and their counts
print(f"Number of Unique Domains that are NOT @enron.com: {len(domain_counts)}")

domain_counts_df = domain_counts.reset_index()
# Rename a column in place
domain_counts_df.rename(columns={0: "Domain", "count": "Count"}, inplace=True)

print("Most common non-Enron email domains:")
domain_counts_df

Number of Unique Domains that are NOT @enron.com: 5280
Most common non-Enron email domains:


,Domain,Count
0,aol.com,1764
1,hotmail.com,1681
2,yahoo.com,966
3,haas.berkeley.edu,731
4,ccomad3.uu.commissioner.com,700
...,...,...
5275,mriresearch.com,1
5276,yahoo.fr,1
5277,energymarketsolutions.com,1
5278,arcadeequipmentwarehouse.com,1


In [78]:
import pandas as pd

# Assuming emails_df is your DataFrame and it's already loaded

# Step 1: Clean the 'from' column
emails_df["from_cleaned"] = (
    emails_df["from"].str.replace(r">$", "", regex=True).str.lower()
)

# Step 2: Filter for Enron emails
enron_emails_df = emails_df[
    emails_df["from_cleaned"].str.endswith("enron.com", na=False)
]

# Step 3: Group by Sender and Employee and count
sender_employee_counts = (
    enron_emails_df.groupby(["from_cleaned", "x-from"])
    .size()
    .reset_index(name="Count")
    .rename(columns={"from_cleaned": "Sender", "x-from": "Employee"})
)

# Optional: Reorder columns
sender_employee_counts = sender_employee_counts[["Sender", "Employee", "Count"]]

# Remove text between '<' and '>', including the angle brackets themselves
sender_employee_counts["Employee"] = (
    sender_employee_counts["Employee"]
    .str.replace(r"<.*?>", "", regex=True)
    .str.strip()  # Remove any leading/trailing whitespace
)

# Split 'Employee' column by comma, reorder, and join as "First Last"
sender_employee_counts["Employee"] = (
    sender_employee_counts["Employee"]
    .str.replace(r"<.*?>", "", regex=True)  # Remove '<...>'
    .str.strip()  # Strip leading/trailing whitespace
    .str.split(", ")  # Split by comma and space
    .apply(
        lambda x: f"{x[1]} {x[0]}" if len(x) == 2 else x[0]
    )  # Reorder to "First Last"
)

# Display the resulting DataFrame
# sender_employee_counts

# Group by 'Sender' and 'Employee' and sum the 'Count'
combined_sender_employee_counts = sender_employee_counts.groupby(
    ["Sender", "Employee"], as_index=False
)["Count"].sum()

# Sort by "Count" in descending order
combined_sender_employee_counts = combined_sender_employee_counts.sort_values(
    by="Count", ascending=False
).reset_index(drop=True)

combined_sender_employee_counts

,Sender,Employee,Count
0,jeff.dasovich@enron.com,Jeff Dasovich,5496
1,kay.mann@enron.com,Kay Mann,4868
2,sara.shackleton@enron.com,Sara Shackleton,4442
3,tana.jones@enron.com,Tana Jones,4152
4,vince.kaminski@enron.com,Vince J Kaminski,3935
...,...,...,...
7170,mark.mcclure@enron.com,Mark McClure,1
7171,clickathome.relations-@enron.com,ClickAtHome and Community Relations-,1
7172,ramona.perkins@enron.com,Ramona Perkins,1
7173,click.home@enron.com,Click at Home,1


In [80]:
# Split 'Employee' column to extract 'First' and 'Last' names
combined_sender_employee_counts[["First", "Last"]] = combined_sender_employee_counts[
    "Employee"
].str.split(" ", n=1, expand=True)

combined_sender_employee_counts

,Sender,Employee,Count,First,Last
0,jeff.dasovich@enron.com,Jeff Dasovich,5496,Jeff,Dasovich
1,kay.mann@enron.com,Kay Mann,4868,Kay,Mann
2,sara.shackleton@enron.com,Sara Shackleton,4442,Sara,Shackleton
3,tana.jones@enron.com,Tana Jones,4152,Tana,Jones
4,vince.kaminski@enron.com,Vince J Kaminski,3935,Vince,J Kaminski
...,...,...,...,...,...
7170,mark.mcclure@enron.com,Mark McClure,1,Mark,McClure
7171,clickathome.relations-@enron.com,ClickAtHome and Community Relations-,1,ClickAtHome,and Community Relations-
7172,ramona.perkins@enron.com,Ramona Perkins,1,Ramona,Perkins
7173,click.home@enron.com,Click at Home,1,Click,at Home


In [82]:
# Use regex to filter names with a middle initial
middle_initial_names = combined_sender_employee_counts[
    combined_sender_employee_counts["Employee"].str.contains(
        r"^\w+\s[A-Z]\s\w+$", na=False
    )
]["Employee"].unique()

# Convert the result to a list
middle_initial_names = list(middle_initial_names)

# Display the list of names with middle initials
print(
    f"Employees with Middle Initials ({len(middle_initial_names)}): {middle_initial_names}"
)

Employees with Middle Initials (404): ['Vince J Kaminski', 'Richard B Sanders', 'Steven J Kean', 'Jeffrey A Shankman', 'Mark E Haedicke', 'Phillip M Love', 'John J Lavorato', 'Darron C Giron', 'David W Delainey', 'Daren J Farmer', 'Dan J Hyvl', 'Susan M Scott', 'Phillip K Allen', 'Susan J Mara', 'James D Steffes', 'Hunter S Shively', 'Stacey W White', 'Patrice L Mims', 'Jane M Tholt', 'Stacy E Dickson', 'Christi L Nicolay', 'Rob G Gay', 'Kevin M Presto', 'Liz M Taylor', 'Randall L Gay', 'Ann M Schmidt', 'Rhonda L Denton', 'Larry F Campbell', 'Chris H Foster', 'Sandra F Brawner', 'Fletcher J Sturm', 'Susan W Pereira', 'Christopher F Calger', 'Jeffrey T Hodge', 'Jeffrey C Gossett', 'Mona L Petrochko', 'Rebecca W Cantrell', 'John M Forney', 'Brenda F Herod', 'Bob M Hall', 'Peter F Keavey', 'Kevin G Moore', 'Jason R Williams', 'Michael E Taylor', 'Lee L Papayoti', 'Susan M Landwehr', 'Robert E Lloyd', 'Yvette G Connevey', 'Steve C Hall', 'Kenny J Soignet', 'Gary W Lamphier', 'William S Bra

In [83]:
# Remove middle initial from names in "Employee" column
combined_sender_employee_counts["Employee"] = combined_sender_employee_counts[
    "Employee"
].str.replace(r"^(\w+)\s[A-Z]\s(\w+)$", r"\1 \2", regex=True)\

combined_sender_employee_counts

,Sender,Employee,Count,First,Last
0,jeff.dasovich@enron.com,Jeff Dasovich,5496,Jeff,Dasovich
1,kay.mann@enron.com,Kay Mann,4868,Kay,Mann
2,sara.shackleton@enron.com,Sara Shackleton,4442,Sara,Shackleton
3,tana.jones@enron.com,Tana Jones,4152,Tana,Jones
4,vince.kaminski@enron.com,Vince Kaminski,3935,Vince,J Kaminski
...,...,...,...,...,...
7170,mark.mcclure@enron.com,Mark McClure,1,Mark,McClure
7171,clickathome.relations-@enron.com,ClickAtHome and Community Relations-,1,ClickAtHome,and Community Relations-
7172,ramona.perkins@enron.com,Ramona Perkins,1,Ramona,Perkins
7173,click.home@enron.com,Click at Home,1,Click,at Home


In [84]:
# Use regex to filter names with a middle initial
middle_initial_names = combined_sender_employee_counts[
    combined_sender_employee_counts["Employee"].str.contains(
        r"^\w+\s[A-Z]\s\w+$", na=False
    )
]["Employee"].unique()

# Convert the result to a list
middle_initial_names = list(middle_initial_names)

# Display the list of names with middle initials
print(
    f"Employees with Middle Initials ({len(middle_initial_names)}): {middle_initial_names}"
)

Employees with Middle Initials (0): []


In [85]:
# Check for duplicates in the 'Sender' column
duplicates_in_sender = combined_sender_employee_counts[
    combined_sender_employee_counts["Sender"].duplicated()
]

# Count the number of duplicates
num_duplicates = len(duplicates_in_sender)

# Display the result
print(f"Number of duplicate 'Sender' entries: {num_duplicates}")
print(duplicates_in_sender[["Sender"]])  # Optional: Display duplicates

Number of duplicate 'Sender' entries: 677
                          Sender
28          pete.davis@enron.com
35          pete.davis@enron.com
61         susan.scott@enron.com
70         mark.taylor@enron.com
100   arsystem@mailman.enron.com
...                          ...
7147            ethink@enron.com
7152      randy.lebeau@enron.com
7164       clickathome@enron.com
7166       clickathome@enron.com
7170      mark.mcclure@enron.com

[677 rows x 1 columns]


In [94]:
employee_df = combined_sender_employee_counts.groupby(
    ["Sender", "Employee"], as_index=False
)["Count"].sum()

# Check for duplicates in the 'Sender' column
employee_duplicates = employee_df[employee_df["Sender"].duplicated()]

# Count the number of duplicates
n_duplicates = len(employee_duplicates)

# Display the result
print(f"Number of duplicate 'Sender' entries: {n_duplicates}")
print(employee_df[["Sender"]])  # Optional: Display duplicates
# Check for duplicates in the 'Sender' and 'Employee' columns
employee_duplicates = employee_df[employee_df[["Sender", "Employee"]].duplicated()]

# Count the number of duplicates
n_duplicates = len(employee_duplicates)

# Display the result
print(f"Number of duplicate 'Sender' and 'Employee' entries: {n_duplicates}")
print(employee_duplicates[["Sender", "Employee"]])  # Optional: Display duplicates

# If you want to view all rows with duplicates in the 'Sender' column only
sender_duplicates = employee_df[employee_df["Sender"].duplicated()]
print("Duplicate 'Sender' rows:")
print(sender_duplicates[["Sender"]])  # Optional: Display Sender duplicates

Number of duplicate 'Sender' entries: 566
                             Sender
0     'todd'.delahoussaye@enron.com
1                   2.ews@enron.com
2                      3e@enron.com
3                   4.ews@enron.com
4                   40ect@enron.com
...                             ...
7059            zimmerman@enron.com
7060     zionette.vincent@enron.com
7061               zipper@enron.com
7062             zufferli@enron.com
7063         zulie.flores@enron.com

[7064 rows x 1 columns]
Number of duplicate 'Sender' and 'Employee' entries: 0
Empty DataFrame
Columns: [Sender, Employee]
Index: []
Duplicate 'Sender' rows:
                                    Sender
5                          40ect@enron.com
7                          40ees@enron.com
8                          40ees@enron.com
9                          40ees@enron.com
11                       40enron@enron.com
...                                    ...
6778      transportation.parking@enron.com
6785             trena.

In [99]:
sender_duplicates.head(50)


,Sender,Employee,Count
5,40ect@enron.com,Tammy Gilmore/NA/Enron@ECT,1
7,40ees@enron.com,Becky Pitre/HOU/EES@EES,2
8,40ees@enron.com,Bill Hatch/HOU/EES@EES,1
9,40ees@enron.com,Product Idea Bank/HOU/EES@EES,1
11,40enron@enron.com,Anne C Koehler/HOU/ECT@ENRON,1
12,40enron@enron.com,Barbara N Gray/HOU/ECT@ENRON,3
13,40enron@enron.com,Barry Pearce/Enron Communications@ENRON COMMUN...,2
14,40enron@enron.com,Beth Apollo- VP Global Risk Management Operati...,1
15,40enron@enron.com,Body Shop@ENRON,1
16,40enron@enron.com,Bruce Garner/LON/ECT@ECT@ENRON,1


In [100]:
sender_duplicates.tail(50)

,Sender,Employee,Count
5484,randy.lebeau@enron.com,Randy Lebeau,1
5496,raul.rizo-patron@enron.com,Raul Rizo-patron,1
5503,raymond.bowen@enron.com,Raymond Bowen Jr.,14
5556,resources@enron.com,human resources@enron.com,4
5577,richard.amabile@enron.com,Richard.Amabile@enron.com,1
5609,richard.lydecker@enron.com,Richard Lydecker Jr.,6
5624,richard.shapiro@enron.com,"Richard"" @ENRON ""Shapiro",1
5675,rob.walls@enron.com,Rob Walls Jr.,56
5710,robert.hemstock@enron.com,Robert.Hemstock@enron.com,1
5719,robert.jones@mailman.enron.com,Robert.Jones@mailman.enron.com; HR VP @ENRON,18


In [ ]:
# Display the entire employee_df
# employee_df

In [107]:
# Assuming emails_df is your DataFrame and it's already loaded

# Step 1: Clean the 'from' column
emails_df["from_cleaned"] = (
    emails_df["from"].str.replace(r">$", "", regex=True).str.lower()
)

# Step 2: Filter for Enron emails
enron_emails_df = emails_df[
    emails_df["from_cleaned"].str.endswith("enron.com", na=False)
]

# Step 3: Group by Sender and Employee and count
employee_emails_sent = (
    enron_emails_df.groupby(["from_cleaned"])
    .size()
    .reset_index(name="Count")
    .rename(columns={"from_cleaned": "Sender"})
)

# Sort by "Count" in descending order
employee_emails_sent = employee_emails_sent.sort_values(
    by="Count", ascending=False
).reset_index(drop=True)

employee_emails_sent.head(50)

,Sender,Count
0,jeff.dasovich@enron.com,5498
1,kay.mann@enron.com,4868
2,sara.shackleton@enron.com,4443
3,tana.jones@enron.com,4152
4,vince.kaminski@enron.com,3935
5,pete.davis@enron.com,3921
6,chris.germany@enron.com,3620
7,matthew.lenhart@enron.com,2762
8,debra.perlingiere@enron.com,2439
9,gerald.nemec@enron.com,2230


In [139]:
# Remove rows where 'Sender' matches the standard first.last@enron.com email format
# The regex matches emails in the form of first.last@enron.com (lowercase)
odd_email_addresses = employee_df[
    ~employee_df["Sender"].str.match(r"^[a-z]+\.[a-z]+@enron\.com$", na=False)
]

# Display the rows with odd email addresses
# print(odd_email_addresses[["Sender"]])

print(f"Number of @enron.com email addresses: {len(employee_df)}")
# Optionally, count how many rows have "odd" email addresses
num_odd_emails = len(odd_email_addresses)
print(f"Number of 'odd' email addresses: {num_odd_emails}")
# odd_email_addresses.head(50)

bad_emails = [
    "2.ews@enron.com",
    "3e@enron.com",
    "4.ews@enron.com",
    "6.ews@enron.com",
    "9.ews@enron.com",
    "40ect@enron.com",
    "40ees@enron.com",
    "40enron@enron.com",
    "ze.powergroup.inc.@mailman.enron.com",
    "weather@enron.com",
    "webrequests@enron.com",
    "westdesksupport@enron.com",
    "actforchange.com@mailman.enron.com",
    "4b98e0e8-e7271b97-862566a9-4fb484@enron.com",
    # "andrew.gieser.enronxgate@enron.com",
    "outlook-migration-team@enron.com",
    "arsystem@mailman.enron.com",
    "perfmgmt@enron.com",
    "mbx_iscinfra@enron.com",
    "ipayit@enron.com",
    "ministryofsound.com@mailman.enron.com",
    "integrated.solutions.center@enron.com",
    "clickathome.relations-@enron.com",
    "investinme@enron.com",
    "megan_la@mailman.enron.com",
    "clickathome@enron.com",
    "ibuyit@enron.com",
    "newport-news.com@mailman.enron.com",
    "ministryofsound.com@mailboy.enron.com",
    "enron.mailsweeper.admin@enron.com",
    "eserver@enron.com",
    "ethink@enron.com",
    "postmaster@enron.com",
    "sap_security@enron.com",
    "legal.7@enron.com",
    "ebiz@enron.com",
    "perfmgmt@ect.enron.com",
    "archiving@enron.com",
    "bodyshop@enron.com",
    "legal.1@enron.com",
    "team.kingman-needles@enron.com",
]

# Remove rows where 'Sender' is in the 'bad_emails' list
cleaned_odd_email_addresses = odd_email_addresses[
    ~odd_email_addresses["Sender"].isin(bad_emails)
]

# Optionally, count how many rows remain after removing the bad emails
num_cleaned_odd_emails = len(cleaned_odd_email_addresses)
print(f"Number of 'odd' email addresses after cleaning: {num_cleaned_odd_emails}")

# Sort by "Count" in descending order
cleaned_odd_email_addresses = cleaned_odd_email_addresses.sort_values(
    by="Count", ascending=False
).reset_index(drop=True)

cleaned_odd_email_addresses.head(5)

Number of @enron.com email addresses: 7064
Number of 'odd' email addresses: 1069
Number of 'odd' email addresses after cleaning: 848


,Sender,Employee,Count
0,d..steffes@enron.com,James D. Steffes,1456
1,m..presto@enron.com,Kevin M. Presto,793
2,houston <.ward@enron.com,Kim S (Houston) Ward,628
3,m..love@enron.com,Phillip M. Love,623
4,m..scott@enron.com,Susan M. Scott,549


In [133]:
# Filter rows where 'Count' is greater than 50
filtered_df = cleaned_odd_email_addresses[cleaned_odd_email_addresses["Count"] < 57]
filtered_df.head(50)
# credit <.williams@enron.com pr <.palmer@enron.com

,Sender,Employee,Count
49,l..denton@enron.com,Rhonda L. Denton,56
50,c..koehler@enron.com,Anne C. Koehler,56
51,m..landwehr@enron.com,Susan M. Landwehr,50
52,n..gray@enron.com,Barbara N. Gray,47
53,s..theriot@enron.com,Kim S. Theriot,47
54,d..winfree@enron.com,O'Neal D. Winfree,45
55,c..hall@enron.com,Steve C. Hall,45
56,credit <.williams@enron.com,Jason R (Credit) Williams,45
57,lavorato@enron.com,John J. Lavorato,43
58,sarah.wesner-soong@enron.com,Sarah Wesner-Soong,40


In [ ]:
# Fix ying <.chen@enron.com
# Fix virginia.o'neill@enron.com
# Lots of w..name with w.. two dots
# "andrew.gieser.enronxgate@enron.com" ?? long email "mike.harper.enronxgate@enron.com"
# houston <.ward@enron.com

# trading <.williams@enron.com Jason (Trading) Williams???
# "ipayit@enron.com" Investigate
# paul.y'barbo@enron.com fix
# egm <.sharp@enron.com fix
# Lots of ministry of sound...
# foreign_froms46.txt@mailman.enron.com???
# enron.corporate.administrative.services@enron.com
